In [39]:
import process_data
from process_data import get_dfs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit   
import os
import importlib
importlib.reload(process_data)


<module 'process_data' from '/Users/xnimir/Desktop/Sn exp 2024/process_data.py'>

In [34]:
def gaussian(x, A, mu, sigma, B):
    return A * np.exp(-((x - mu)**2) / (2 * sigma**2)) + B

In [35]:
def fitting(x,y):
    x_fit = np.linspace(np.min(x), np.max(x), 500)

    initial_guess = [np.max(y), x[np.argmax(y)], np.std(x), 0]
    bounds = ([0, 0, 0, 0], [np.inf, np.max(x), np.inf, np.inf])
    popt, pcov = curve_fit(gaussian, x, y, p0=initial_guess, bounds=bounds, sigma=np.sqrt(y), absolute_sigma=True)
    y_fit = gaussian(x_fit, *popt)
    # print(initial_guess)
    # print(popt)

    return x_fit, y_fit, popt, pcov

In [36]:
def get_scatter_plot(data_df, freq_df, file):
    x = data_df['Bin center']
    y = data_df['Count raw']
    x_fit, y_fit, popt, pcov = fitting(x, y)

    fwhm = 2.355 * popt[2] * 1E6
    fit_errors = np.sqrt(np.diag(pcov))
    mu_error = fit_errors[1]

    plt.errorbar(x, y, yerr=np.sqrt(y), fmt='o', capsize=3)
    plt.plot(x_fit, y_fit)
    plt.scatter(freq_df['Bin center'], freq_df['Count raw'], color='red')
    plt.title(f'{file}')
    plt.xlabel('frequency (THz)')
    plt.ylabel('binned counts')

    label = f'mu={popt[1]:.6f} THz\nfwhm={fwhm:.2f} MHz'
    plt.text(0.95, 0.95, label, fontsize=10,
                        verticalalignment='top', horizontalalignment='right',
                        transform=plt.gca().transAxes) 
    plt.show()

    return popt[1], fwhm

In [37]:
def main(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.tdms'):
            scaled_df, time_df, freq_df = get_dfs(folder_path)
            get_scatter_plot(scaled_df, freq_df, filename)

In [40]:
def single_set():
    folder_path = '/Users/xnimir/Desktop/Sn exp 2024/data/set3/'
    main(folder_path)
single_set()

processing,  20241024_1848_Sn-120_ref2_set3.tdms
processing,  20241024_1656_Sn-120_set3_ref1.tdms
processing,  20241024_1730_Sn-116_set3.tdms
processing,  20241024_2309_Sn-120_ref5_set3.tdms


ValueError: too many values to unpack (expected 3)